# Asistente turístico de Tenerife (Entrega Final - LLMs)

Este notebook implementa la **entrega final de la asignatura Large Language Models (LLMs)**.  
El objetivo es desarrollar un **asistente conversacional turístico** que combine:

- **RAG (Retrieval-Augmented Generation)** sobre la guía turística de Tenerife (`data/TENERIFE.pdf`).
- **Diálogo multiturno** con memoria y recorte de contexto para mantener coherencia entre turnos.
- **Function calling** obligatorio: `get_weather(fecha, lugar)` → devuelve la predicción del tiempo (simulada en este proyecto) con gestión de errores y logs.
- **Documentación y reproducibilidad**: todo el flujo está en este notebook, con celdas de código, resultados y explicaciones en Markdown.

---

## Estructura del notebook

1. **Carga de librerías y configuración**  
   - Rutas del proyecto, `.env`, logs.  

2. **Procesamiento del PDF y creación del índice (RAG)**  
   - Lectura y *chunking*.  
   - Generación de embeddings y almacenamiento en Chroma.  
   - Verificación del número de páginas y chunks.  

3. **Prueba de recuperación**  
   - Búsqueda de ejemplo en el vector store.  
   - Revisión de resultados y citaciones `[doc: página]`.  

4. **Respuestas con LLM + RAG**  
   - Integración de recuperación y generación.  
   - Respuestas con citas a la fuente.  

5. **Diálogo multiturno con memoria**  
   - Historial de conversación.  
   - Recorte por tokens para no superar límites del modelo.  

6. **Function calling `get_weather`**  
   - Definición con Pydantic (JSON Schema).  
   - Simulación determinista de resultados.  
   - Manejo de errores y registro en logs.  

7. **Evaluación y análisis**  
   - Conjunto de prompts reproducibles.  
   - Métricas simples (ej. recall heurístico).  
   - Discusión de limitaciones y mejoras.  

---

In [49]:
from pathlib import Path
from dotenv import load_dotenv
from loguru import logger
import os, json

# Rutas del proyecto
ROOT = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_PDF = ROOT / "data" / "TENERIFE.pdf"
DB_DIR = ROOT / "chroma_db"
LOG_DIR = ROOT / "logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)
logger.add(LOG_DIR / "assistant.log", rotation="1 MB")

# Cargar variables (.env)
load_dotenv(ROOT / ".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

assert DATA_PDF.exists(), f"No encuentro {DATA_PDF}. Asegúrate de poner TENERIFE.pdf en /data"
assert OPENAI_API_KEY, "Falta OPENAI_API_KEY en .env"
print("Rutas OK, .env OK")

Rutas OK, .env OK


In [50]:
from pypdf import PdfReader

def read_pdf_text(path: Path):
    reader = PdfReader(str(path))
    pages = []
    for i, p in enumerate(reader.pages, start=1):
        txt = (p.extract_text() or "").strip()
        pages.append((i, txt))
    return pages

def chunk_text(text: str, max_chars: int = 900, overlap: int = 150):
    chunks = []
    start = 0
    N = len(text)
    while start < N:
        end = min(N, start + max_chars)
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk)
        if end == N: break
        start = max(0, end - overlap)
    return chunks

pages = read_pdf_text(DATA_PDF)
print(f"Paginas leídas: {len(pages)}")

docs, metadatas = [], []
for page_num, txt in pages:
    for i, c in enumerate(chunk_text(txt, max_chars=900, overlap=150), start=1):
        docs.append(c)
        metadatas.append({"page": page_num, "chunk_id": f"p{page_num}_c{i}"})
len(docs), len(metadatas)

Paginas leídas: 25


(33, 33)

In [51]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

DB_DIR.mkdir(exist_ok=True, parents=True)

embedding_fn = SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

client = chromadb.PersistentClient(path=str(DB_DIR))
collection = client.get_or_create_collection(
    name="tenerife",
    metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_fn
)

# Si está vacío, añadimos documentos
if collection.count() == 0:
    collection.add(
        documents=docs,
        metadatas=metadatas,
        ids=[m["chunk_id"] for m in metadatas],
    )

print("Documentos indexados:", collection.count())

Documentos indexados: 33


## RAG: preparación del índice

- **Fuente**: `data/TENERIFE.pdf` (25 páginas).
- **Chunking**: 900 caracteres con solape de 150 → equilibrio entre contexto suficiente y precisión en la búsqueda.
- **Embeddings**: modelo `sentence-transformers/all-MiniLM-L6-v2` (rápido y eficiente).
- **Vector store**: Chroma persistente en `chroma_db/` con métrica coseno.
- **Resultado**: 25 páginas procesadas → 33 chunks indexados.

Este paso nos asegura que el asistente podrá buscar de manera semántica en la guía y recuperar fragmentos relevantes para responder con citas.

In [52]:
def retrieve(query: str, k: int = 4):
    res = collection.query(query_texts=[query], n_results=k)
    items = list(zip(res["ids"][0], res["documents"][0], res["metadatas"][0], res["distances"][0]))
    return items

query = "Miradores recomendados en el Parque rural de Anaga"
hits = retrieve(query, k=4)
for id_, doc, meta, dist in hits:
    print(f"[doc: {meta['page']}] {id_}  score={1-dist:.3f}\n{doc[:300]}...\n")

[doc: 5] p5_c1  score=0.658
• Parque rural de Anaga 
Es un espacio natural protegido, declarado Reserva de la Biosfera y que cuenta con la 
mayor cantidad de endemismos de Europa. 
Tiene bastantes miradores chulísimos y rutas que podéis hacer si os mola este tipo 
de planes y si teníais pensado alguna caminata (véase Las Mejor...

[doc: 6] p6_c1  score=0.486
o Mirador Pico del Inglés (aquí tenéis una de las fotos típicas de Anaga, en un 
pasillo que hay super frondoso) [ubicación]...

[doc: 21] p21_c2  score=0.465
 ballenas 
(recomendado). También podéis hacer excursiones en kayak, motos de agua y esas 
vainas. Para consultar información sobre este tipo de excursiones, visitad este enlace 
o buscad vosotros mismos que seguro que encontráis un montón de actividades al 
llegar a Los Gigantes....

[doc: 25] p25_c1  score=0.440
▪ Arepera La Carajita (entre La Orotava y el Puerto de la Cruz 
[ubicación]. Este sitio tiene las mejores arepas que podréis comer en 
vuestra vida y son baratísima

## Prueba de recuperación

Consulta realizada: **“Miradores recomendados en el Parque rural de Anaga”**

El sistema devuelve varios fragmentos del PDF junto con la referencia `[doc: página]`.  
Revisando el contenido, confirmamos que los resultados corresponden con la guía turística (ej. descripción del Parque rural de Anaga, Mirador Pico del Inglés, etc.).

Esto valida que el índice funciona correctamente y que los fragmentos recuperados serán útiles para generar respuestas con citas en el siguiente paso.

## Respuestas con LLM + RAG (turno único)

Integramos la recuperación (Chroma) con un LLM comercial. El asistente debe **citar las fuentes** en el formato `[doc: página]` usando únicamente el contexto recuperado.

In [53]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)  # usa la clave del .env

def format_context(items):
    blocks = []
    for _id, doc, meta, _dist in items:
        blocks.append(f"[doc:{meta['page']}] {doc}")
    return "\n\n".join(blocks)

SYSTEM_PROMPT = (
    "Eres un asistente turístico de Tenerife.\n"
    "Responde en español y **cita las fuentes** usando el formato [doc: página].\n"
    "Usa solo el contexto proporcionado; si falta información, dilo explícitamente.\n"
)

In [54]:
def answer_with_rag(question: str, k: int = 4, temperature: float = 0.3):
    items = retrieve(question, k=k)
    context = format_context(items)
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Pregunta: {question}\n\nContexto:\n{context}"}
    ]
    resp = client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        messages=messages,
        temperature=temperature,
        top_p=0.95,
        max_tokens=500,
    )
    return resp.choices[0].message.content


In [55]:
print(answer_with_rag("¿Qué miradores recomiendas en Anaga y por qué?"))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}